In [1]:
from tkinter import messagebox
from tkinter import *
from tkinter import simpledialog
import tkinter
from tkinter import filedialog
from tkinter.filedialog import askopenfilename
import time
import cv2
import tensorflow as tf
from collections import namedtuple
from collections import defaultdict
from io import StringIO
from PIL import Image
import numpy as np
import winsound

main = tkinter.Tk()
main.title("Accident Identification")
main.geometry("1300x1200")

global filename
global detectionGraph
global msg

def loadModel():
    global detectionGraph
    detectionGraph = tf.Graph()
    with detectionGraph.as_default():
        od_graphDef = tf.compat.v1.GraphDef()
        with tf.compat.v2.io.gfile.GFile('model/frozen_inference_graph.pb', 'rb') as file:
            serializedGraph = file.read()
            od_graphDef.ParseFromString(serializedGraph)
            tf.import_graph_def(od_graphDef, name='')
    
    messagebox.showinfo("Training model loaded","Training model loaded")
    
def beep():
    frequency = 2500  # Set Frequency To 2500 Hertz
    duration = 500  # Set Duration To 1000 ms == 1 second
    winsound.Beep(frequency, duration)
    
def uploadVideo():
    global filename
    filename = filedialog.askopenfilename(initialdir="videos")
    pathlabel.config(text=filename)
    text.delete('1.0', END)
    text.insert(END,filename+" loaded\n");

def calculateCollision(boxes,classes,scores,image_np):
    global msg
    #cv2.putText(image_np, "NORMAL!", (230, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2, cv2.LINE_AA)
    for i, b in enumerate(boxes[0]):
        if classes[0][i] == 3 or classes[0][i] == 6 or classes[0][i] == 8:
            if scores[0][i] > 0.73:
                for j, c in enumerate(boxes[0]):
                    if (i != j) and (classes[0][j] == 3 or classes[0][j] == 6 or classes[0][j] == 8) and scores[0][j]> 0.5:
                        Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')
                        ra = Rectangle(boxes[0][i][3], boxes[0][i][2], boxes[0][i][1], boxes[0][i][3])
                        rb = Rectangle(boxes[0][j][3], boxes[0][j][2], boxes[0][j][1], boxes[0][j][3])
                         # Accessing attributes of rectangles
                        print("Rectangle A: xmin={}, ymin={}, xmax={}, ymax={}".format(ra.xmin, ra.ymin, ra.xmax, ra.ymax))
                        print("Rectangle B: xmin={}, ymin={}, xmax={}, ymax={}".format(rb.xmin, rb.ymin, rb.xmax, rb.ymax))

                        ar = rectArea(boxes[0][i][3], boxes[0][i][1],boxes[0][i][2],boxes[0][i][3])
                        col_threshold = 0.6*np.sqrt(ar)
                        area(ra, rb)
                        if (area(ra,rb)<col_threshold) :
                            print('accident')
                            msg = 'ACCIDENT!'
                            beep()
                            return True
                        else:
                            return False
                        
def rectArea(xmax, ymax, xmin, ymin):
    x = np.abs(xmax-xmin)
    y = np.abs(ymax-ymin)
    return x*y

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

def area(a, b):  # returns None if rectangles don't intersect
    dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    return dx*dy

def detector():
    global msg
    msg = ''
    cap = cv2.VideoCapture(filename)
    with detectionGraph.as_default():
        with tf.compat.v1.Session(graph=detectionGraph) as sess:
            while True:
                ret, image_np = cap.read()
                image_np_expanded = np.expand_dims(image_np, axis=0)
                image_tensor = detectionGraph.get_tensor_by_name('image_tensor:0')
                boxes = detectionGraph.get_tensor_by_name('detection_boxes:0')
                scores = detectionGraph.get_tensor_by_name('detection_scores:0')
                classes = detectionGraph.get_tensor_by_name('detection_classes:0')
                num_detections = detectionGraph.get_tensor_by_name('num_detections:0')
                if image_np_expanded.shape is None:
                    # Handle the case when image_np_expanded is None
                    return
                image_np_expanded = image_np_expanded.astype(np.float32)
                
                (boxes, scores, classes, num_detections) = sess.run([boxes, scores, classes, num_detections], feed_dict={image_tensor: image_np_expanded})
                print("Boxes",boxes)
                print("Scores",scores)
                print("Classes",classes)
                calculateCollision(boxes, classes, scores, image_np)
                cv2.putText(image_np, msg, (230, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 2, cv2.LINE_AA)
                cv2.imshow('Accident Identification', image_np)
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    cv2.destroyAllWindows()
                    break

def exit():
    main.destroy()
    
font = ('times', 16, 'bold')
title = Label(main, text='Accident Identification')
title.config(bg='light cyan', fg='pale violet red')  
title.config(font=font)           
title.config(height=3, width=120)       
title.place(x=0,y=5)

font1 = ('times', 13, 'bold')
uploadButton = Button(main, text="Load & Generate CNN Model", command=loadModel)
uploadButton.place(x=50,y=100)
uploadButton.config(font=font1)  

pathlabel = Label(main)
pathlabel.config(bg='light cyan', fg='pale violet red')  
pathlabel.config(font=font1)           
pathlabel.place(x=460,y=100)

webcamButton = Button(main, text="Browse System Videos", command=uploadVideo)
webcamButton.place(x=50,y=150)
webcamButton.config(font=font1)

webcamButton = Button(main, text="Start Accident Detector", command=detector)
webcamButton.place(x=50,y=200)
webcamButton.config(font=font1) 

exitButton = Button(main, text="Exit", command=exit)
exitButton.place(x=330,y=250)
exitButton.config(font=font1) 

font1 = ('times', 12, 'bold')
text=Text(main,height=20,width=150)
scroll=Scrollbar(text)
text.configure(yscrollcommand=scroll.set)
text.place(x=10,y=250)
text.config(font=font1)

main.config(bg='snow3')
main.mainloop()

C:\Users\TEJASWI\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Boxes [[[0.37540936 0.05340299 0.5309088  0.7508136 ]
  [0.4910042  0.7226079  0.5836046  0.80638355]
  [0.4910042  0.7226079  0.5836046  0.80638355]
  [0.38240445 0.1683858  0.4302897  0.2145158 ]
  [0.4731231  0.7180696  0.5875385  0.95324   ]
  [0.37962672 0.20601302 0.43221816 0.2495867 ]
  [0.3827333  0.08885336 0.4233933  0.10430664]
  [0.48551592 0.72662365 0.5885599  0.88298774]
  [0.9769351  0.25718936 0.9947514  0.31137517]
  [0.07038845 0.8496922  0.47682726 0.9968823 ]
  [0.3021975  0.7409862  0.49286458 0.97214425]
  [0.08463053 0.93262905 0.47514325 0.9983404 ]
  [0.23784344 0.77986145 0.50862134 0.9548563 ]
  [0.47859144 0.01675138 0.97605896 0.98969007]
  [0.4410003  0.58693653 0.4754502  0.623216  ]
  [0.9764362  0.20852467 0.9963061  0.26833168]
  [0.30217987 0.0235334  0.55757624 0.45866838]
  [0.5189561  0.83919126 0.71249944 0.9961762 ]
  [0.5245088  0.74817836 0.6464216  1.        ]
  [0.61178845 0.97387457 0.88657266 0.9920995 ]
  [0.7793966  0.26602444 0.800175 

Boxes [[[0.40387678 0.         0.5314451  0.858017  ]
  [0.44937745 0.69452536 0.4798161  0.73225045]
  [0.44679722 0.52418584 0.4842486  0.57281846]
  [0.17730945 0.8691374  0.21058655 0.8819456 ]
  [0.44090766 0.58958435 0.47136813 0.6263653 ]
  [0.49311548 0.7247074  0.5842925  0.80865663]
  [0.3824042  0.0897396  0.42191565 0.10351831]
  [0.44685146 0.6533707  0.4758928  0.6891753 ]
  [0.39148104 0.55005616 0.42140228 0.56399816]
  [0.4827213  0.15310122 0.5211746  0.22020786]
  [0.4702367  0.7215963  0.5557927  0.8098758 ]
  [0.6738876  0.6896007  0.69639003 0.7119845 ]
  [0.45352128 0.51111597 0.5245214  0.60240716]
  [0.37752932 0.50770706 0.41677266 0.5301141 ]
  [0.6283161  0.9725265  0.84595126 0.99292463]
  [0.44357175 0.         0.60235924 0.50755835]
  [0.48130292 0.19109407 0.5209691  0.28174585]
  [0.00454395 0.00301856 0.21174586 0.17759162]
  [0.49311548 0.7247074  0.5842925  0.80865663]
  [0.6067152  0.9553166  0.9874768  0.99907047]
  [0.67850816 0.68925554 0.6961769

Boxes [[[1.71404943e-01 8.69794488e-01 2.07959428e-01 8.82620692e-01]
  [4.43422794e-01 5.32092690e-01 4.84517097e-01 5.73211551e-01]
  [4.91946578e-01 7.25809693e-01 5.83049536e-01 8.10022831e-01]
  [3.98850083e-01 1.02735758e-02 5.23604274e-01 8.38891506e-01]
  [4.67820585e-01 7.21500158e-01 5.59921205e-01 8.09408903e-01]
  [4.91946578e-01 7.25809693e-01 5.83049536e-01 8.10022831e-01]
  [5.30191898e-01 8.45720530e-01 5.94583035e-01 9.83023047e-01]
  [4.48929906e-01 5.07029533e-01 5.33012569e-01 6.09092593e-01]
  [3.98822933e-01 1.33594684e-02 4.29899246e-01 6.36248440e-02]
  [4.14288044e-03 1.51999295e-03 2.06102639e-01 1.79371417e-01]
  [4.45130527e-01 6.99364662e-01 4.77958202e-01 7.29972959e-01]
  [4.76471096e-01 6.94035292e-01 5.30765653e-01 7.46629477e-01]
  [4.80726451e-01 7.24773526e-01 5.99082470e-01 9.78400469e-01]
  [4.80593115e-01 7.30614901e-01 5.88268578e-01 8.78994942e-01]
  [6.38182878e-01 9.77813423e-01 8.31758142e-01 9.91953552e-01]
  [3.02111387e-01 7.05839455e-01 5

Boxes [[[0.49119222 0.72505003 0.58387303 0.8074253 ]
  [0.3932261  0.         0.526164   0.85216177]
  [0.46721864 0.72294736 0.56173456 0.8026296 ]
  [0.16803405 0.87310064 0.2097418  0.88800955]
  [0.44699252 0.5344923  0.48513967 0.5724185 ]
  [0.48190945 0.728884   0.5891673  0.88034904]
  [0.44811577 0.704054   0.47767776 0.73155355]
  [0.29850656 0.7053136  0.49512464 0.94557256]
  [0.47609156 0.69587827 0.53134495 0.7477274 ]
  [0.47835782 0.7254406  0.5997088  0.97396135]
  [0.49119222 0.72505003 0.58387303 0.8074253 ]
  [0.45008075 0.50554705 0.5310893  0.6080377 ]
  [0.76939607 0.8500096  0.7910472  0.9144327 ]
  [0.3972757  0.01101944 0.42974043 0.06301472]
  [0.00405216 0.2910527  0.39590472 0.3940484 ]
  [0.45756036 0.74346864 0.5678248  0.89328635]
  [0.00312753 0.00146619 0.20277669 0.17823192]
  [0.2679907  0.7321484  0.5646712  0.901134  ]
  [0.17359155 0.9468303  0.50657094 0.99513984]
  [0.6191164  0.9849801  0.86443627 0.99435365]
  [0.46389976 0.9497051  0.5580438

...

Boxes [[[4.29484904e-01 8.55169833e-01 6.81603730e-01 9.93355930e-01]
  [2.84041345e-01 5.45882285e-02 3.69552732e-01 7.95297772e-02]
  [2.09529877e-01 5.00701480e-02 3.67140055e-01 8.43370259e-02]
  [3.70386392e-01 8.93229842e-02 5.32141566e-01 3.23221654e-01]
  [7.94062316e-02 6.90653920e-03 5.73599219e-01 3.44778657e-01]
  [2.99722731e-01 1.68483585e-01 5.15402138e-01 3.15243810e-01]
  [4.43219244e-01 2.19629705e-03 5.49650550e-01 5.00353277e-02]
  [1.72095597e-02 7.71784782e-03 8.37934613e-01 2.75351822e-01]
  [3.70233506e-01 9.32612777e-01 4.34409469e-01 9.50521111e-01]
  [4.48824942e-01 9.31073010e-01 7.22024143e-01 9.99000371e-01]
  [2.89867640e-01 1.02369808e-01 5.27081072e-01 3.05294305e-01]
  [4.81521070e-01 8.51860106e-01 6.69884384e-01 9.03107226e-01]
  [2.26169914e-01 5.39704114e-02 5.38206100e-01 3.29765379e-01]
  [5.41067660e-01 3.97799969e-01 5.65280259e-01 4.32912350e-01]
  [1.26114875e-01 1.18048593e-01 5.64939380e-01 5.32368422e-01]
  [2.88431764e-01 2.50072122e-01 5

Boxes [[[2.84167022e-01 5.46470694e-02 3.69289011e-01 7.94694722e-02]
  [2.09623069e-01 5.00263721e-02 3.66879374e-01 8.44127387e-02]
  [4.45359081e-01 2.12070532e-03 5.49646258e-01 4.99617606e-02]
  [3.69837493e-01 8.83608758e-02 5.31628251e-01 3.22808057e-01]
  [2.99702406e-01 1.67941540e-01 5.14828205e-01 3.14970762e-01]
  [8.12442601e-02 7.61707127e-03 5.74202418e-01 3.45198154e-01]
  [9.02694464e-03 2.76218235e-01 6.83365345e-01 9.97247398e-01]
  [1.96177661e-02 7.63532519e-03 8.39118838e-01 2.70037442e-01]
  [2.87803173e-01 1.02728121e-01 5.26683390e-01 3.04450065e-01]
  [2.26579517e-01 5.50746173e-02 5.38361073e-01 3.30460727e-01]
  [1.37286782e-02 0.00000000e+00 6.57460749e-01 7.02948570e-01]
  [0.00000000e+00 4.55018252e-01 9.12956238e-01 9.95652318e-01]
  [1.26049563e-01 1.22625023e-01 5.66348732e-01 5.31521916e-01]
  [5.17005324e-02 2.43406415e-01 5.83849967e-01 4.79074657e-01]
  [4.09179926e-02 7.33756185e-01 7.81662941e-01 9.96686697e-01]
  [3.04368854e-01 2.39755377e-01 4

Boxes [[[2.85152286e-01 5.48377857e-02 3.69055778e-01 7.94407800e-02]
  [2.09918767e-01 4.98523414e-02 3.67111713e-01 8.43181610e-02]
  [4.46232200e-01 2.15088949e-03 5.49435675e-01 4.99730371e-02]
  [7.63324499e-01 8.84083629e-01 8.04532409e-01 9.30404425e-01]
  [2.99854845e-01 1.69208944e-01 5.14699697e-01 3.15060377e-01]
  [3.69707584e-01 8.89865011e-02 5.31763792e-01 3.24480057e-01]
  [8.26096833e-02 7.06510246e-03 5.71063042e-01 3.45387220e-01]
  [7.70816505e-01 8.86986196e-01 8.21485460e-01 9.29745376e-01]
  [2.88035393e-01 1.03526615e-01 5.27223647e-01 3.03777546e-01]
  [2.10507214e-02 7.32462108e-03 8.28224778e-01 2.69344628e-01]
  [0.00000000e+00 3.93425792e-01 8.67468357e-01 1.00000000e+00]
  [2.29516417e-01 5.56484610e-02 5.38060784e-01 3.32620680e-01]
  [7.70816505e-01 8.86986196e-01 8.21485460e-01 9.29745376e-01]
  [1.33813024e-02 0.00000000e+00 6.56490326e-01 7.03351498e-01]
  [3.03340316e-01 2.39113316e-01 4.34430480e-01 2.77181685e-01]
  [1.31610870e-01 1.22024074e-01 5

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\TEJASWI\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\TEJASWI\AppData\Local\Temp\ipykernel_5904\2686244432.py", line 108, in detector
    (boxes, scores, classes, num_detections) = sess.run([boxes, scores, classes, num_detections], feed_dict={image_tensor: image_np_expanded})
  File "C:\Users\TEJASWI\anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 969, in run
    result = self._run(None, fetches, feed_dict, options_ptr,
  File "C:\Users\TEJASWI\anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1192, in _run
    results = self._do_run(handle, final_targets, final_fetches,
  File "C:\Users\TEJASWI\anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1372, in _do_run
    return self._do_call(_run_fn, feeds, fetches, targets, options,
  File "C:\Users\TEJASWI\anaconda3\lib\site-packages\tens

KeyboardInterrupt: 